In [ ]:
# Import modules
#Environment -- geopy
import matplotlib.pyplot as plt
import pandas as pd
import os
import requests

In [ ]:
from scipy.stats import linregress

import numpy as np
import pandas as pd
from category_encoders import GLMMEncoder
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import statsmodels.api as sm
from statsmodels.compat import lzip


In [ ]:
xls = pd.ExcelFile('USA_NO2_MSA_GDP.xlsx')
msa = pd.read_excel(xls, 'MSA_2010')
#pop_2019 = pd.read_excel(xls, '2019')
pop_2010 =pd.read_excel(xls, '2010')
us = pd.read_excel(xls,'US_13')
m_19=pd.read_excel(xls,'Sheet4')
full = pd.read_excel(xls, 'usa')

msa_full =msa.merge(m_19[['Merge',2019]],how='left',left_on='Label',right_on='Merge')

In [ ]:
#Select just the 2019 and 2010 years for of the US dataset

us_19= full.query("Year ==2019").copy().drop_duplicates().set_index('CityCountry')
us_10= full.query("Year ==2010").copy().drop_duplicates().set_index('CityCountry')

us_19['Slope']= full.groupby('CityCountry').apply(lambda v: linregress(v.Year, v.NO2)[0])


In [ ]:
NW = ['WA','OR','ID']
SW=['CA','NV','UT','NM','AZ','CO']
NGP=['MT','WY','ND','SD','NE']
SGP=['TX','OK','KS']
SE=['LA','AR','AL','FL','MS','TN','KY','GA','SC','NC','DC','VA']
NE = ['WV','PA','NY','MD','DE','NH','VT','ME','MA','RI','NJ','CT']
MW =['WI','MO','MN','MI','IN','IL','IA','OH']
Other = ['AK','HI']

rdict = {'NW':NW,'SW':SW,'NGP':NGP,'SGP':SGP,'SE':SE,'NE':NE,'MW':MW,'Other':Other}

In [ ]:
#Rename values to merge into single DC and merge with GDP
merge= us_19.reset_index().merge(gdp.reset_index(),how='left')
merge.set_index('CityCountry',inplace=True)
merge['Base_GDP'] =merge[2010]
merge['GDP'] =merge[2019]
merge['Base_NO2']=us_10['NO2']
merge['Base_Population']=us_10['Population']
merge['NO2T']=np.log(merge.NO2)
merge['BaseT']=np.sqrt(merge.Base_NO2)
merge['LogGDP']=np.log(merge[2010])
gdp = pd.read_excel(xls,'GDP')

gdp.set_index('CityCountry', inplace=True)

In [ ]:
merge['Region']=''
for i in rdict:
    filter = merge["State"].isin(rdict[i])
    merge['Region'].where(~filter,i,inplace=True)

#Create metrics from merged datafile

In [ ]:
##Percapita metric

pcap=msa_full.merge(gdp,how='right',on='CityCountry')
pcap['Base_PerCapita']=pcap['2010_y']/pcap['2010_x']
pcap['PerCapita']=pcap['2019_y']/pcap['2019_x']
pcap.set_index('CityCountry',inplace=True)

merge['PerCapita']=pcap['PerCapita']


In [ ]:
#Percent change metric
merge['ConcPercChange']=100*(merge.NO2-merge.Base_NO2)/merge.Base_NO2

In [ ]:
y= plot['NO2T']
X = plot[['BaseT','LogGDP','Region']]
enc = GLMMEncoder(cols=['Region']).fit(X,y)
numeric_dataset = enc.transform(X)
numeric_dataset

In [ ]:
X2 = sm.add_constant(numeric_dataset)
est = sm.OLS(y, X2)
est2 = est.fit()
est2.summary()


In [ ]:
from scipy.stats.mstats import zscore
zx =zscore(numeric_dataset)
zy = zscore(y)
#zx2 = sm.add_constant(zx)
model = sm.OLS(zy,zx)
results = model.fit()
results.params

In [ ]:
from sklearn import linear_model

def calculate_stats(dataframe):
    dataframe['Predicted']=np.exp(predicted)
    dataframe['Actual']= np.exp(actual)
    dataframe['pc_dif']=100*(dataframe.Actual-dataframe.Predicted)/dataframe.Actual
    dataframe['abs_dif']= dataframe['pc_dif'].abs()
    dataframe['Residuals']= dataframe.Actual-dataframe.Predicted


regr = linear_model.LinearRegression(fit_intercept=True)
regr.fit(numeric_dataset, y)
coef = regr.coef_
#coef = np.polyfit(x, y, z, 1)
inter = regr.intercept_
actual = []
predicted = []
for ind in range(len(numeric_dataset)):
    actual.append(y.iloc[ind])
    predicted.append(sum([coef[i]*numeric_dataset.iloc[ind][i] for i in range(len(coef))])+inter)
    

##predicted is predicted NO2/capita
calculate_stats(merge)


In [ ]:
#Dictionary to conver the US state to its abbreviation for merging
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [ ]:
asthma= pd.read_csv('asthmaincidence/IHME-GBD_2019_DATA-bd31445e-1.csv')
asthma['location_name']=asthma.location_name.replace(us_state_to_abbrev)
asthma

In [ ]:
# Sample air quality data (replace this with your own data)
plot = merge

# Define weights for each metric (adjust these weights as needed)
metrics = {'ConcPercChange':'ConcChange','Slope':'slope','Residuals':'resid'}

# Normalize the data using min-max scaling
for metric in metrics:
    min_val = plot[metric].min()
    max_val = plot[metric].max()
    plot[metrics[metric] + '_Score'] = (plot[metric] - min_val) / (max_val - min_val)

# Calculate scores for each city

# Calculate total scores for each city
plot['Total_Score'] = plot[['ConcChange_Score', 'slope_Score','resid_Score']].sum(axis=1)/3

# Rank cities based on total scores
plot = plot.sort_values(by='Total_Score', ascending=False)

In [ ]:
plot=plot.merge(asthma.query('year==2010')[['location_name','val']],how='left',left_on='State',right_on='location_name').drop_duplicates().rename(columns={'val':'asthma_2010'})
plot=plot.merge(asthma.query('year==2019')[['location_name','val']],how='left',left_on='State',right_on='location_name').drop_duplicates().rename(columns={'val':'asthma_2019'})

In [ ]:
plot['ActualAF']=plot.Actual.apply(calculate_af)
plot['PredictedAF']=plot.Predicted.apply(calculate_af)
plot['BaseAF'] =plot.Base_NO2.apply(calculate_af)
plot['Actual_Cases'] = plot.ActualAF*(plot.asthma_2019/1e5)*plot.Pop_ped
plot['Predicted_Cases'] = plot.PredictedAF*(plot.asthma_2019/1e5)*plot.Pop_ped
plot['Base_Cases'] = plot.BaseAF*(plot.asthma_2010/1e5)*plot.Base_Pop_ped
plot['Cases_Residual']=plot.Actual_Cases-plot.Predicted_Cases
plot['Rates_Residual'] = plot['Cases_Residual']/(plot.Pop_ped/1e5)

In [ ]:
plot.to_csv('Inputs/us_asthmaprediction.csv')